In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import os
import numpy as np

In [2]:
from shapely.geometry import Point, Polygon

In [3]:
sns.set(style="darkgrid")

In [4]:
filepath = os.path.join("Resources", "data.csv")
data = pd.read_csv(filepath)
data.head()

,neighbourhood
0,Waterfront Communities-The Island
1,Newtonbrook East
2,York University Heights
3,Dorset Park
4,Englemount-lawrence


In [5]:
nb = os.path.join('Resources', 'Neighbourhoods', "Neighbourhoods.shp")
regions = gpd.read_file(nb)
regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()
regions.sample(5)

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_14,FIELD_15,geometry,neighbourhood
133,2234,25886882,25926795,49885,90,90,Junction Area (90),Junction Area (90),None,None,-79.471440,43.667890,16493633,5.033628e+06,10771.397082,"POLYGON ((-79.48263 43.66683, -79.48272 43.667...",junction area
106,2207,25886903,25926768,49885,93,93,Dovercourt-Wallace Emerson-Junction (93),Dovercourt-Wallace Emerson-Junction (93),None,None,-79.438541,43.665677,16493201,7.125378e+06,12750.544329,"POLYGON ((-79.41842 43.66358, -79.41887 43.663...",dovercourt-wallace emerson-junction
125,2226,25886455,25926787,49885,134,134,Highland Creek (134),Highland Creek (134),None,None,-79.177472,43.790775,16493505,1.007702e+07,13732.929447,"POLYGON ((-79.17527 43.78021, -79.17535 43.780...",highland creek
123,2224,25886941,25926785,49885,88,88,High Park North (88),High Park North (88),None,None,-79.466302,43.657565,16493473,3.598782e+06,8733.411724,"POLYGON ((-79.45245 43.65632, -79.45382 43.656...",high park north
103,2204,25886353,25926765,49885,59,59,Danforth East York (59),Danforth East York (59),None,None,-79.331403,43.689468,16493153,4.188624e+06,10540.912802,"POLYGON ((-79.34627 43.68272, -79.34645 43.683...",danforth east york


In [6]:
regions.to_csv('Resources/regions.csv')

In [7]:
starbucks_df = pd.read_csv('Resources/Starbucks locations.csv')
starbucks_df.head()

,ID,Lat,Long
0,75921-104040,43.086574,-79.059356
1,3997-146205,43.077276,-79.082792
2,75525-35359,43.078906,-79.081879
3,75633-86381,43.079703,-79.082204
4,75790-96396,43.082189,-79.082469


In [8]:
# Create a set of starbucks locations lat and lng combinations
lng_lats = []

lngs = starbucks_df['Long']
lats = starbucks_df['Lat']

lng_lats = zip(lngs, lats)

neighbourhood_id = []
neighbourhood_name = []
lats_list = []
lngs_list = []

for lng_lat in lng_lats:
    point = Point(lng_lat[0], lng_lat[1])
    
    for i in np.arange(len(regions)):
        poly = regions.loc[i, 'geometry']
        
        if point.within(poly):
            neighbourhood_id.append(regions.loc[i, 'FIELD_6'])
            neighbourhood_name.append(regions.loc[i, 'neighbourhood'])
            lats_list.append(lng_lat[1])
            lngs_list.append(lng_lat[0])

In [9]:
starbucks_dict = {
    'Lat' : lats_list,
    'Long' : lngs_list,
    'Hood ID' : neighbourhood_id,
    'Neighbourhood' : neighbourhood_name
}

clean_starbucks = pd.DataFrame(starbucks_dict)

clean_starbucks

,Lat,Long,Hood ID,Neighbourhood
0,43.797471,-79.148805,131,rouge
1,43.743753,-79.216334,139,scarborough village
2,43.770374,-79.186485,136,west hill
3,43.816306,-79.293317,130,milliken
4,43.776324,-79.259292,127,bendale
...,...,...,...,...
173,43.766139,-79.476335,27,york university heights
174,43.726773,-79.451830,31,yorkdale-glen park
175,43.757568,-79.465829,27,york university heights
176,43.725807,-79.450222,31,yorkdale-glen park


In [10]:
# number of stores for each location
starbucks_val_counts = clean_starbucks['Neighbourhood'].value_counts()
starbucks_val_counts = starbucks_val_counts.reset_index()
starbucks_val_counts.columns = ['Neighbourhood', 'Number of Stores']

starbucks_val_counts

,Neighbourhood,Number of Stores
0,bay street corridor,26
1,waterfront communities-the island,19
2,church-yonge corridor,8
3,islington-city centre west,8
4,annex,7
...,...,...
67,tam o'shanter-sullivan,1
68,kingsview village-the westway,1
69,old east york,1
70,trinity-bellwoods,1


In [11]:
clean_starbucks_df = clean_starbucks.drop_duplicates(subset='Neighbourhood', keep='first')
clean_starbucks_df

,Lat,Long,Hood ID,Neighbourhood
0,43.797471,-79.148805,131,rouge
1,43.743753,-79.216334,139,scarborough village
2,43.770374,-79.186485,136,west hill
3,43.816306,-79.293317,130,milliken
4,43.776324,-79.259292,127,bendale
...,...,...,...,...
164,43.698454,-79.463205,108,briar hill-belgravia
167,43.665399,-79.471892,90,junction area
169,43.651357,-79.475875,88,high park north
170,43.725315,-79.454165,31,yorkdale-glen park


In [12]:
starbucks_final = pd.merge(starbucks_val_counts, clean_starbucks_df, on='Neighbourhood', how='inner')
starbucks_final.drop(['Lat', 'Long'], axis='columns', inplace=True)

In [13]:
starbucks_final = starbucks_final[['Hood ID','Neighbourhood','Number of Stores']]
starbucks_final.rename(columns={"Hood ID": "Hood_ID"}, inplace=True)

In [14]:
starbucks_final

,Hood_ID,Neighbourhood,Number of Stores
0,76,bay street corridor,26
1,77,waterfront communities-the island,19
2,75,church-yonge corridor,8
3,14,islington-city centre west,8
4,95,annex,7
...,...,...,...
67,118,tam o'shanter-sullivan,1
68,6,kingsview village-the westway,1
69,58,old east york,1
70,81,trinity-bellwoods,1


In [15]:
starbucks_final.to_csv('output_data/starbucks_final.csv')